# Udacity - Machine Learning Engineer Nanodegree
## Capstone Project
### Title: Development of a LSTM Network to Predict Students’ Answers on Exam Questions

### Implementation of DKT:
#### Part 1: Define constants

In [1]:
fn = "data/ASSISTments_skill_builder_data.csv" # Dataset path
verbose = 1 # Verbose = {0,1,2}
best_model_weights = "weights/bestmodel" # File to save the model.
log_dir = "logs" # File to save the training log.
optimizer = "rmsprop" # Optimizer to use
lstm_units = 100 # Number of LSTM units
batch_size = 16 # Batch size
epochs = 3 # Number of epochs to train
dropout_rate = 0.3 # Dropout rate
test_fraction = 0.2 # Portion of data to be used for testing
validation_fraction = 0.2 # Portion of training data to be used for validation

#### Part 2: Pre-processing

In [2]:
from deepkt import deepkt, data_util, metrics


dataset, length, nb_features, nb_skills = data_util.load_dataset(fn=fn,
                                                                 batch_size=batch_size,
                                                                 shuffle=True)

train_set, test_set, val_set = data_util.split_dataset(dataset=dataset,
                                                       total_size=length,
                                                       test_fraction=test_fraction,
                                                       val_fraction=validation_fraction)


set_sz = length * batch_size
test_set_sz = (set_sz * test_fraction)
val_set_sz = (set_sz - test_set_sz) * validation_fraction
train_set_sz = set_sz - test_set_sz - val_set_sz
print("============= Data Summary =============")
print("Total number of students: %d" % set_sz)
print("Training set size: %d" % train_set_sz)
print("Validation set size: %d" % val_set_sz)
print("Testing set size: %d" % test_set_sz)
print("Number of skills: %d" % nb_skills)
print("Number of features in the input: %d" % nb_features)
print("========================================")

============= Data Summary =============
Total number of students: 4160
Training set size: 2662
Validation set size: 665
Testing set size: 832
Number of skills: 123
Number of features in the input: 246


#### Part 3: Building the model

In [3]:
student_model = deepkt.DKTModel(
                        nb_features=nb_features,
                        nb_skills=nb_skills,
                        hidden_units=lstm_units,
                        dropout_rate=dropout_rate)

student_model.compile(
        optimizer=optimizer,
        metrics=[
            metrics.BinaryAccuracy(),
            metrics.AUC(),
            metrics.Precision(),
            metrics.Recall()
        ])

student_model.summary()

Model: "DKTModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, None, 246)]       0         
_________________________________________________________________
masking (Masking)            (None, None, 246)         0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 100)         138800    
_________________________________________________________________
dropout (TimeDistributed)    (None, None, 100)         0         
_________________________________________________________________
outputs (TimeDistributed)    (None, None, 123)         12423     
Total params: 151,223
Trainable params: 151,223
Non-trainable params: 0
_________________________________________________________________


#### Part 4: Train the Model

In [5]:
import tensorflow as tf
history = student_model.fit(dataset=train_set,
                            epochs=epochs,
                            verbose=verbose,
                            validation_data=val_set,
                            callbacks=[ 
                                tf.keras.callbacks.CSVLogger(f"{log_dir}/train.log"),
                                tf.keras.callbacks.ModelCheckpoint(best_model_weights,
                                                                   save_best_only=True,
                                                                   save_weights_only=True),
                                tf.keras.callbacks.TensorBoard(log_dir=log_dir)
        ])

Epoch 1/3
167/167 [==============================] - 11s 68ms/step - loss: 0.0855 - binary_accuracy: 0.7945 - auc: 0.8389 - precision: 0.7893 - recall: 0.9589 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/3
167/167 [==============================] - 8s 50ms/step - loss: 0.0421 - binary_accuracy: 0.9363 - auc: 0.9868 - precision: 0.9234 - recall: 0.9892 - val_loss: 0.0184 - val_binary_accuracy: 0.9846 - val_auc: 0.9991 - val_precision: 0.9804 - val_recall: 0.9982
Epoch 3/3
167/167 [==============================] - 8s 50ms/step - loss: 0.0106 - binary_accuracy: 0.9916 - auc: 0.9996 - precision: 0.9897 - recall: 0.9984 - val_loss: 0.0048 - val_binary_accuracy: 0.9983 - val_auc: 0.9999 - val_precision: 0.9983 - val_recall: 0.9991


#### Part 5: Load the Model with the Best Validation Loss

In [6]:
student_model.load_weights(best_model_weights)

#### Part 6: Test the Model

In [7]:
result = student_model.evaluate(test_set, verbose=verbose)

     52/Unknown - 1s 28ms/step - loss: 0.0047 - binary_accuracy: 0.9934 - auc: 0.9999 - precision: 0.9917 - recall: 0.9988